<a href="https://colab.research.google.com/github/ecemuzman/RFM_Analysis/blob/main/FLO_RFM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer Segmentation with RFM

---

## Business Problem
FLO aims to segment its customers and develop marketing strategies tailored to each segment.  
To achieve this goal, customer behaviors will be analyzed, and clusters will be created based on these behavioral patterns.

---

## Dataset Story
The dataset consists of information obtained from the past shopping behaviors of customers who made their last purchases between **2020 and 2021** through **OmniChannel** (both online and offline) platforms.

### Variables:
- **master_id:** Unique customer ID  
- **order_channel:** The platform or channel used for the purchase (Android, iOS, Desktop, Mobile, Offline)  
- **last_order_channel:** The channel used for the most recent purchase  
- **first_order_date:** Date of the customer’s first purchase  
- **last_order_date:** Date of the customer’s most recent purchase  
- **last_order_date_online:** Date of the most recent purchase on the online platform  
- **last_order_date_offline:** Date of the most recent purchase on the offline platform  
- **order_num_total_ever_online:** Total number of purchases made online  
- **order_num_total_ever_offline:** Total number of purchases made offline  
- **customer_value_total_ever_offline:** Total amount spent on offline purchases  
- **customer_value_total_ever_online:** Total amount spent on online purchases  
- **interested_in_categories_12:** List of categories in which the customer made purchases in the last 12 months  


---

## Tasks
1. Data understanding and preprocessing  
2. Calculation of RFM metrics  
3. Creating RFM segments  
4. Interpreting the segments and developing marketing strategies  

---


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt

In [34]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [10]:
df_ = pd.read_csv("/content/flo_data_20k.csv")
df = df_.copy()

In [11]:
df.head(10)


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.000,2.000,150.870,49.990,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.000,1.000,59.990,315.940,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.000,1.000,49.990,113.640,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.000,2.000,120.480,934.210,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.000,1.000,69.980,95.980,"[KADIN, AKTIFSPOR]"


In [12]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


In [13]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [14]:
df.isnull().sum()


,0
master_id,0
order_channel,0
last_order_channel,0
first_order_date,0
last_order_date,0
last_order_date_online,0
last_order_date_offline,0
order_num_total_ever_online,0
order_num_total_ever_offline,0
customer_value_total_ever_offline,0


### 3. Omnichannel customers are those who shop from both online and offline platforms.  
Create new variables for each customer’s **total number of purchases** and **total spending**.


In [15]:
df["omni_channel_order_num"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["omni_channel_customer_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
print(df.head())

                              master_id order_channel last_order_channel  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   

  first_order_date last_order_date last_order_date_online  \
0       2020-10-30      2021-02-26             2021-02-21   
1       2017-02-08      2021-02-16             2021-02-16   
2       2019-11-27      2020-11-27             2020-11-27   
3       2021-01-06      2021-01-17             2021-01-17   
4       2019-08-03      2021-03-07             2021-03-07   

  last_order_date_offline  order_num_total_ever_online  \
0              2021-02-26                        4.000   
1              2020-01-10                       19.000   
2 

In [16]:
date_cols = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
df[date_cols] = df[date_cols].apply(pd.to_datetime)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [17]:
flo_describe = df.groupby("order_channel").agg({'master_id': lambda x: x.nunique(),
                                                'omni_channel_order_num': lambda x: x.sum(),
                                                'omni_channel_customer_value': lambda x: x.sum()})

print(flo_describe)

               master_id  omni_channel_order_num  omni_channel_customer_value
order_channel                                                                
Android App         9495               52269.000                  7819062.760
Desktop             2735               10920.000                  1610321.460
Ios App             2833               15351.000                  2525999.930
Mobile              4882               21679.000                  3028183.160


In [18]:
df.sort_values(by='omni_channel_order_num', ascending=False).head(10)


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,omni_channel_order_num,omni_channel_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.000,45905.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.000,12282.240
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.000,109.000,10239.460,143.980,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.000,10383.440
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.000,87.000,8432.250,139.980,[ERKEK],88.000,8572.230
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.000,81.000,3997.550,242.810,[ERKEK],83.000,4240.360
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.000,33918.100
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.000,74.000,4640.770,543.280,"[KADIN, AKTIFSPOR]",77.000,5184.050
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.000,3.000,292.930,5004.950,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.000,5297.880
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",70.000,18443.570
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.000,16918.570


In [19]:
df.sort_values(by='omni_channel_customer_value', ascending=False).head(10)


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,omni_channel_order_num,omni_channel_customer_value
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.000,45905.100
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.000,1.000,130.490,36687.800,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.000,36818.290
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.000,33918.100
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.000,1.000,538.940,30688.470,"[ERKEK, KADIN, AKTIFSPOR]",11.000,31227.410
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.000,3.000,18119.140,2587.200,[AKTIFSPOR],4.000,20706.340
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",70.000,18443.570
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.000,16918.570
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.000,1.000,180.730,12545.370,"[ERKEK, AKTIFSPOR]",37.000,12726.100
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.000,12282.240
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.000,2.000,64.970,12038.180,"[ERKEK, KADIN]",20.000,12103.150


## Calculation of RFM Metrics

RFM (Recency, Frequency, Monetary) is a marketing analysis technique used to evaluate and segment customers based on their purchasing behavior.

- **Recency (R):** How recently the customer made their last purchase.  
- **Frequency (F):** How often the customer makes purchases.  
- **Monetary (M):** How much money the customer spends in total.

By calculating these three metrics for each customer, we can understand their engagement level and purchasing value — which helps in creating targeted marketing strategies.


In [20]:
today_date = df["last_order_date"].max() + dt.timedelta(days=2)


In [28]:
rfm = df.groupby("master_id").agg({'last_order_date': lambda date : (today_date - date).dt.days,
                                   'omni_channel_order_num': lambda x: x,
                                   'omni_channel_customer_value': lambda num: num.sum()})
print(rfm.head())

                                      last_order_date  omni_channel_order_num  \
master_id                                                                       
00016786-2f5a-11ea-bb80-000d3a38a36f               10                   5.000   
00034aaa-a838-11e9-a2fc-000d3a38a36f              298                   3.000   
000be838-85df-11ea-a90b-000d3a38a36f              213                   4.000   
000c1fe2-a8b7-11ea-8479-000d3a38a36f               27                   7.000   
000f5e3e-9dde-11ea-80cd-000d3a38a36f               20                   7.000   

                                      omni_channel_customer_value  
master_id                                                          
00016786-2f5a-11ea-bb80-000d3a38a36f                      776.070  
00034aaa-a838-11e9-a2fc-000d3a38a36f                      269.470  
000be838-85df-11ea-a90b-000d3a38a36f                      722.690  
000c1fe2-a8b7-11ea-8479-000d3a38a36f                      874.160  
000f5e3e-9dde-11ea-80cd-

In [30]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']
print(rfm.head())

                                      Recency  Frequency  Monetary
master_id                                                         
00016786-2f5a-11ea-bb80-000d3a38a36f       10      5.000   776.070
00034aaa-a838-11e9-a2fc-000d3a38a36f      298      3.000   269.470
000be838-85df-11ea-a90b-000d3a38a36f      213      4.000   722.690
000c1fe2-a8b7-11ea-8479-000d3a38a36f       27      7.000   874.160
000f5e3e-9dde-11ea-80cd-000d3a38a36f       20      7.000  1620.330


In [31]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,19945.000,134.458,103.281,2.000,43.000,111.000,202.000,367.000
Frequency,19945.000,5.025,4.743,2.000,3.000,4.000,6.000,202.000
Monetary,19945.000,751.244,895.402,44.980,339.980,545.270,897.780,45905.100


In [32]:
rfm["Recency_score"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["Frequency_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["Monetary_score"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])


In [35]:
rfm["RF_SCORE"] = (rfm["Recency_score"].astype(str) + rfm["Frequency_score"].astype(str))
print(rfm.head())

                                      Recency  Frequency  Monetary  \
master_id                                                            
00016786-2f5a-11ea-bb80-000d3a38a36f       10      5.000   776.070   
00034aaa-a838-11e9-a2fc-000d3a38a36f      298      3.000   269.470   
000be838-85df-11ea-a90b-000d3a38a36f      213      4.000   722.690   
000c1fe2-a8b7-11ea-8479-000d3a38a36f       27      7.000   874.160   
000f5e3e-9dde-11ea-80cd-000d3a38a36f       20      7.000  1620.330   

                                     Recency_score Frequency_score  \
master_id                                                            
00016786-2f5a-11ea-bb80-000d3a38a36f             5               4   
00034aaa-a838-11e9-a2fc-000d3a38a36f             1               2   
000be838-85df-11ea-a90b-000d3a38a36f             2               3   
000c1fe2-a8b7-11ea-8479-000d3a38a36f             5               4   
000f5e3e-9dde-11ea-80cd-000d3a38a36f             5               4   

                  

Defining Customer Segments Using RF Scores


In [38]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['Segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
print(rfm.head())

                                      Recency  Frequency  Monetary  \
master_id                                                            
00016786-2f5a-11ea-bb80-000d3a38a36f       10      5.000   776.070   
00034aaa-a838-11e9-a2fc-000d3a38a36f      298      3.000   269.470   
000be838-85df-11ea-a90b-000d3a38a36f      213      4.000   722.690   
000c1fe2-a8b7-11ea-8479-000d3a38a36f       27      7.000   874.160   
000f5e3e-9dde-11ea-80cd-000d3a38a36f       20      7.000  1620.330   

                                     Recency_score Frequency_score  \
master_id                                                            
00016786-2f5a-11ea-bb80-000d3a38a36f             5               4   
00034aaa-a838-11e9-a2fc-000d3a38a36f             1               2   
000be838-85df-11ea-a90b-000d3a38a36f             2               3   
000c1fe2-a8b7-11ea-8479-000d3a38a36f             5               4   
000f5e3e-9dde-11ea-80cd-000d3a38a36f             5               4   

                  

In [39]:
#Example
rfm[rfm["Segment"] == "cant_loose"].index


Index(['007cdfe4-1f54-11ea-87bf-000d3a38a36f',
       '00f6c98e-abb5-11e9-a2fc-000d3a38a36f',
       '00f7553c-5bbb-11ea-b88d-000d3a38a36f',
       '017bd900-a97e-11e9-a2fc-000d3a38a36f',
       '01defd1e-a582-11e9-a2fc-000d3a38a36f',
       '02a59d28-aa20-11e9-a2fc-000d3a38a36f',
       '02d44576-a596-11e9-a2fc-000d3a38a36f',
       '04f12e56-b206-11e9-89fa-000d3a38a36f',
       '0515c070-9e6c-11e9-9897-000d3a38a36f',
       '056e803c-a378-11e9-a2fc-000d3a38a36f',
       ...
       'fd7e5126-ac7e-11e9-a2fc-000d3a38a36f',
       'fd969980-a9d5-11e9-a2fc-000d3a38a36f',
       'fdbe8304-a7ab-11e9-a2fc-000d3a38a36f',
       'fe2863f6-a97b-11e9-a2fc-000d3a38a36f',
       'fe5c3d2c-aa14-11e9-a2fc-000d3a38a36f',
       'fe98b79e-acac-11e9-a2fc-000d3a38a36f',
       'fea8e3dc-af6b-11e9-a2fc-000d3a38a36f',
       'ff0e2222-a81a-11e9-a2fc-000d3a38a36f',
       'ff2ba51e-a9cc-11e9-a2fc-000d3a38a36f',
       'ffce3990-26be-11ea-aaaa-000d3a38a36f'],
      dtype='object', name='master_id', length=1

## ACTION TIME!

### 1. Analyze the average Recency, Frequency, and Monetary values of each segment.

By examining these averages, we can better understand the behavioral characteristics of each customer segment.

In [40]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])


Recency       Frequency       Monetary      
                       mean count      mean count     mean count
Segment                                                         
about_to_sleep      114.032  1643     2.407  1643  361.649  1643
at_Risk             242.329  3152     4.470  3152  648.325  3152
cant_loose          235.159  1194    10.717  1194 1481.652  1194
champions            17.142  1920     8.965  1920 1410.709  1920
hibernating         247.426  3589     2.391  3589  362.583  3589
loyal_customers      82.558  3375     8.356  3375 1216.257  3375
need_attention      113.037   806     3.739   806  553.437   806
new_customers        17.976   673     2.000   673  344.049   673
potential_loyalists  36.870  2925     3.311  2925  533.741  2925
promising            58.695   668     2.000   668  334.153   668

Using the RFM analysis, identify customers that match the following two cases and export their IDs as CSV files.

a. New Brand Target Group

FLO is introducing a new women’s shoe brand, with prices above the general customer preference range.
For marketing and product promotion, FLO wants to directly reach loyal and high-value customers who are most likely to be interested in this new brand.

Target profile:

Belongs to segments: “Champions” or “Loyal Customers”

Average spending greater than 250

Purchased from the Women category

Export the customer IDs of this group to a CSV file named:
new_brand_target_customer_ids.csv

In [43]:
# Merge RFM metrics with the main dataset to allow filtering across all columns


flo_ = rfm.merge(df, on="master_id")
print(flo_.head())

                              master_id  Recency  Frequency  Monetary  \
0  00016786-2f5a-11ea-bb80-000d3a38a36f       10      5.000   776.070   
1  00034aaa-a838-11e9-a2fc-000d3a38a36f      298      3.000   269.470   
2  000be838-85df-11ea-a90b-000d3a38a36f      213      4.000   722.690   
3  000c1fe2-a8b7-11ea-8479-000d3a38a36f       27      7.000   874.160   
4  000f5e3e-9dde-11ea-80cd-000d3a38a36f       20      7.000  1620.330   

  Recency_score Frequency_score Monetary_score RF_SCORE      Segment  \
0             5               4              4       54    champions   
1             1               2              1       12  hibernating   
2             2               3              4       23      at_Risk   
3             5               4              4       54    champions   
4             5               4              5       54    champions   

  order_channel last_order_channel first_order_date last_order_date  \
0        Mobile             Mobile       2019-11-19      

In [50]:
target = flo_[(flo_["Monetary"] > 250) & (flo_["interested_in_categories_12"].str.contains("KADIN", case=False, na=False)) & (flo_["Segment"].isin(["champions", "loyal_customers"]))]
print(target.head())

                               master_id  Recency  Frequency  Monetary  \
3   000c1fe2-a8b7-11ea-8479-000d3a38a36f       27      7.000   874.160   
18  003903e0-abce-11e9-a2fc-000d3a38a36f       68      5.000   577.470   
22  004bec18-9f7a-11e9-a2fc-000d3a38a36f       99     12.000   684.910   
25  00534fe4-a6b4-11e9-a2fc-000d3a38a36f       63     16.000   800.590   
49  009293fe-1f3e-11ea-87bf-000d3a38a36f      129      6.000   725.180   

   Recency_score Frequency_score Monetary_score RF_SCORE          Segment  \
3              5               4              4       54        champions   
18             4               4              3       44  loyal_customers   
22             3               5              4       35  loyal_customers   
25             4               5              4       45  loyal_customers   
49             3               4              4       34  loyal_customers   

   order_channel last_order_channel first_order_date last_order_date  \
3    Android App    

In [48]:
target.to_csv("new_brand_target_customer_ids.csv")


b. Discount Campaign Target Group

A 40% discount is planned for Men’s and Kids’ product categories.
FLO wants to re-engage valuable past customers who have not purchased for a while —
those who are at risk of churning or currently inactive but have shown interest in these categories before.

Target profile:

Interested in Men or Kids categories

Belongs to segments such as “About to Sleep”, “At Risk”, or “New Customers”

Export the customer IDs of this group to a CSV file named:
discount_target_customer_ids.csv

In [51]:
target_ = flo_[(flo_["interested_in_categories_12"].str.contains("ERKEK | COCUK", case=False, na=False)) & (flo_["Segment"].isin(["about_to_sleep", "about_to_sleep", "new_customers"]))]
print(target_.head())

                                master_id  Recency  Frequency  Monetary  \
399  04937794-508e-11ea-99ba-000d3a38a36f       85      3.000   343.950   
455  055f4316-ab32-11ea-b736-000d3a38a36f       96      2.000   217.460   
509  05f83906-9707-11eb-8c2a-000d3a38a36f       27      2.000   229.750   
527  06548210-bd28-11ea-aa61-000d3a38a36f      140      2.000   200.480   
713  08cd7574-b03b-11e9-9757-000d3a38a36f      124      3.000   396.450   

    Recency_score Frequency_score Monetary_score RF_SCORE         Segment  \
399             3               2              2       32  about_to_sleep   
455             3               1              1       31  about_to_sleep   
509             5               1              1       51   new_customers   
527             3               1              1       31  about_to_sleep   
713             3               2              2       32  about_to_sleep   

    order_channel last_order_channel first_order_date last_order_date  \
399   Android

In [47]:
target_.to_csv("discount_target_customer_ids.csv")
